# Training a classifier for weights of author vision components

The features are the distance of the author to any tweet in the conversation
indicated by the following structures:
- subtree to viewed tweet from a tweet the author wrote
- root closeness of viewed tweet
- time delta to viewed tweet from tweets the author wrote


In [25]:
import pandas as pd

df = pd.read_pickle("data/vision_graph_data.pkl")

df.describe()


,timedelta,root_distance_0,y,current,beam_node,has_followed_path,has_follow_path,reply_distance_2,reply_distance_3,root_distance_1,...,root_distance_31,root_distance_32,root_distance_33,root_distance_34,root_distance_35,root_distance_36,root_distance_37,root_distance_38,root_distance_39,root_distance_40
count,3.419848e+06,3.419848e+06,3.419848e+06,3.419848e+06,3.419848e+06,3419848.0,3419848.0,3.419848e+06,3.419848e+06,3.419848e+06,...,3.419848e+06,3.419848e+06,3.419848e+06,3.419848e+06,3.419848e+06,3.419848e+06,3.419848e+06,3.419848e+06,3.419848e+06,3.419848e+06
mean,4.273964e+04,3.969972e-02,5.875466e-02,1.296090e+18,1.296024e+18,0.0,0.0,6.033017e-03,3.545772e-03,1.295455e-01,...,8.772320e-06,5.848213e-06,5.263392e-06,4.678571e-06,4.093749e-06,2.046875e-06,1.754464e-06,1.462053e-06,1.169643e-06,8.772320e-07
std,1.113101e+06,1.952528e-01,2.351650e-01,5.465191e+17,5.464306e+17,0.0,0.0,7.743785e-02,5.944073e-02,3.358028e-01,...,2.961798e-03,2.418301e-03,2.294203e-03,2.162996e-03,2.023298e-03,1.430689e-03,1.324561e-03,1.209153e-03,1.081500e-03,9.366064e-04
min,1.000000e-03,0.000000e+00,0.000000e+00,3.880000e+02,3.880000e+02,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.417000e+03,0.000000e+00,0.000000e+00,1.493856e+18,1.493481e+18,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,9.485000e+03,0.000000e+00,0.000000e+00,1.531777e+18,1.531699e+18,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.239100e+04,0.000000e+00,0.000000e+00,1.539141e+18,1.539003e+18,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.148715e+08,1.000000e+00,1.000000e+00,1.552563e+18,1.552563e+18,0.0,0.0,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [26]:

# remove non-features
current = df.current
beam_node = df.beam_node
platform = df.platform

platform.value_counts()

twitter    2904740
reddit      514793
delab          315
Name: platform, dtype: int64

In [27]:
df = df[df["platform"] == "reddit"]
# df = df[df["root_distance_0"] == 0]

df = df.drop(["current", "beam_node", "platform", "has_followed_path", "has_follow_path"], axis=1)

# normalize timedelta (put between 0 and 1)
dt = df.timedelta
timedelta_normalized = (dt - dt.min()) / (dt.max() - dt.min())
df.timedelta = timedelta_normalized
df

,timedelta,root_distance_0,y,reply_distance_2,reply_distance_3,root_distance_1,root_distance_2,root_distance_3,reply_distance_4,root_distance_4,...,root_distance_31,root_distance_32,root_distance_33,root_distance_34,root_distance_35,root_distance_36,root_distance_37,root_distance_38,root_distance_39,root_distance_40
1815,0.000009,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1816,0.000030,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1817,0.000022,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1818,0.000076,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1819,0.000067,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3418447,0.000106,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3418448,0.000105,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3418449,0.000104,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3418450,0.000102,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
from sklearn.model_selection import train_test_split

# selecting train and test datasets
train, test = train_test_split(df, test_size=0.2)
train.describe()


,timedelta,root_distance_0,y,reply_distance_2,reply_distance_3,root_distance_1,root_distance_2,root_distance_3,reply_distance_4,root_distance_4,...,root_distance_31,root_distance_32,root_distance_33,root_distance_34,root_distance_35,root_distance_36,root_distance_37,root_distance_38,root_distance_39,root_distance_40
count,411834.000000,411834.000000,411834.000000,411834.000000,411834.000000,411834.000000,411834.000000,411834.000000,411834.000000,411834.000000,...,411834.000000,411834.000000,411834.000000,411834.000000,411834.000000,411834.000000,411834.000000,411834.000000,411834.000000,411834.000000
mean,0.000626,0.047274,0.120095,0.024775,0.014115,0.396383,0.220931,0.115945,0.008401,0.058516,...,0.000019,0.000017,0.000024,0.000017,0.000015,0.000012,0.000007,0.000007,0.000007,0.000005
std,0.008741,0.212224,0.325072,0.155438,0.117965,0.489146,0.414875,0.320159,0.091274,0.234717,...,0.004407,0.004123,0.004928,0.004123,0.003817,0.003484,0.002699,0.002699,0.002699,0.002204
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000033,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000117,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000317,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.999997,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [29]:
# train the model
y = train.y
x = train.drop("y", axis=1)
print(x.shape)
print(y.shape)


(411834, 81)
(411834,)


In [30]:
# convert train dataset to numpy arrays not sure this is neded
#x = x.to_numpy()
#y = y.to_numpy()
#print(x[:5])
#print(y[:5])

In [31]:
# import tensorflow and train the model
import tensorflow as tf

print(tf.__version__)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

input_shape = (x.shape[1],)
model = Sequential([
    Dense(1, activation='sigmoid', input_shape=input_shape)
])

# stachistical gradient descend as a classifier seem appropriate
model.compile(
    optimizer='sgd',
    loss='binary_crossentropy',
    metrics=['accuracy', 'mae']
)

model.fit(x, y, epochs=3)


2.6.0
Epoch 1/3
12870/12870 [==============================] - 15s 1ms/step - loss: 0.3444 - accuracy: 0.8795 - mae: 0.2093

In [32]:
# evaluate the model on the test set
test_y = test.y
test_x = test.drop("y", axis=1)

In [33]:
# convert train dataset to numpy arrays not sure this is neded
# test_x = test_x.to_numpy()
# test_y = test_y.to_numpy()

In [34]:
loss, accuracy, mae = model.evaluate(test_x, test_y)
print("the accuracy on the training set is {} and the mae is {}".format(accuracy, mae))

3218/3218 [==============================] - 3s 814us/step - loss: 0.3153 - accuracy: 0.8986 - mae: 0.1816

In [35]:
# have a look at some prediction
reply_distance_2 = test[test["reply_distance_2"] == 1]
reply_distance_2 = reply_distance_2.drop("y", axis=1)
first_rows = reply_distance_2.head(2)
print(first_rows)
model.predict(first_rows)

         timedelta  root_distance_0  reply_distance_2  reply_distance_3  \
733826    0.000167                0               1.0               0.0   
1996962   0.000031                0               1.0               0.0   

         root_distance_1  root_distance_2  root_distance_3  reply_distance_4  \
733826               1.0              0.0              0.0               0.0   
1996962              1.0              0.0              0.0               0.0   

         root_distance_4  reply_distance_5  ...  root_distance_31  \
733826               0.0               0.0  ...               0.0   
1996962              0.0               0.0  ...               0.0   

         root_distance_32  root_distance_33  root_distance_34  \
733826                0.0               0.0               0.0   
1996962               0.0               0.0               0.0   

         root_distance_35  root_distance_36  root_distance_37  \
733826                0.0               0.0               0.0   

array([[0.04649165],
       [0.04649243]], dtype=float32)

In [36]:
# let's have a look at the weights and biases of the hidden layer

first_layer_weights = model.layers[0].get_weights()[0]
first_layer_biases  = model.layers[0].get_weights()[1]
print(first_layer_weights)
column_names = x.columns.values
for i in range(len(column_names)):
    print("feature {} has weight {} \n" .format(column_names[i], first_layer_weights[i]))



[[-1.30677640e-01]
 [ 2.28805804e+00]
 [-8.17832470e-01]
 [ 1.75102639e+00]
 [-3.48071933e-01]
 [-8.52820277e-01]
 [-3.75360668e-01]
 [-2.20548511e-01]
 [-4.99415219e-01]
 [ 7.12498069e-01]
 [ 2.46124957e-02]
 [ 3.55230629e-01]
 [-3.06165874e-01]
 [ 3.23293597e-01]
 [-4.90460247e-02]
 [-6.18301705e-02]
 [-2.22053051e-01]
 [-5.44616431e-02]
 [-1.05244607e-01]
 [-2.05770984e-01]
 [ 1.37624115e-01]
 [ 1.08357072e-01]
 [ 7.39288181e-02]
 [ 1.42271906e-01]
 [ 2.40855590e-01]
 [ 1.89153805e-01]
 [ 1.54219568e-01]
 [ 3.55165899e-02]
 [ 2.46617526e-01]
 [-2.18688861e-01]
 [ 3.05153906e-01]
 [ 2.08589792e-01]
 [-1.57899633e-01]
 [ 1.16328463e-01]
 [-8.16077143e-02]
 [ 3.67255025e-02]
 [-5.92536367e-02]
 [ 1.20465748e-01]
 [ 1.56796187e-01]
 [ 1.99125886e-01]
 [ 2.12041065e-01]
 [ 2.50672549e-01]
 [ 1.05474249e-01]
 [ 1.06483497e-01]
 [-2.41165981e-01]
 [ 1.34024948e-01]
 [ 2.44135305e-01]
 [-9.08605661e-03]
 [-1.88152909e-01]
 [-1.66622818e-01]
 [ 2.15231627e-01]
 [-1.11456305e-01]
 [ 1.1887212